In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config

stparkpap_pap_en_name = "-".join(config.scrapeTarget["languages"])
stparkpap_pap_en = pd.read_csv(f"../data/stparkpap/{stparkpap_pap_en_name}.csv", na_filter=False)
stparkpap_pap_en = stparkpap_pap_en[["pap-simple", "en-simple"]]
stparkpap_pap_en


In [ ]:
import re
sentence_pairs = zip(stparkpap_pap_en["pap-simple"].values, stparkpap_pap_en["en-simple"].values)

associations = {}
for p, e in list(sentence_pairs)[:]:

    p = re.sub('[!?,]', '', p)
    e = re.sub('[!?,]', '', e)

    p_words = p.split()
    e_words = e.split()

    
    for i, p_word in enumerate(p_words):
        current_associations = associations.get(p_word, {})

        if i > len(e_words) - 1:
            continue
        elif i == len(p_words) - 1:
            matching_e = " ".join(e_words[i:])
        else:
            matching_e = e_words[i]
        
        if matching_e in current_associations:
            current_associations[matching_e] += 1
        else:
            current_associations[matching_e] = 1
        
        current_associations =  {k: v for k, v in sorted(current_associations.items(), key=lambda item: item[1], reverse=True)}
        associations[p_word] = current_associations
    # if len(p_words):
associations

In [ ]:
important_associations = {}

for k, v in associations.items():
    frequencies = [frequency for word, frequency in v.items()]
    average_frequency = sum(frequencies) / len(frequencies)
    important_associations[k] = {word:frequency for word, frequency in v.items() if frequency >= average_frequency}

important_associations

In [ ]:
from loader import Loader
from spellcheck import Spellcheck

load = Loader()
lan = "PAP-EN"
dictionary, dictionary_corpus = load.loadDictionary(lan)
# print(dictionary)
spell = Spellcheck(spellchecker_corpus=dictionary_corpus, lan=lan, load=True)

new_associations = {}

for k, v in important_associations.items():
    translation_exists = spell.trie.find(k.lower())
    if translation_exists or k.lower() in config.word_extraction_ignore["stparkpap"]:
        print(f"Skipping {k}")
    else:
        new_associations[k] = list(v.keys())[0]

print(f"Found: {len(new_associations)} Skipped: {len(important_associations)-len(new_associations)} Total: {len(important_associations)}")

In [ ]:
new_associations
new_associations_df = pd.DataFrame(list(new_associations.items()), columns=['pap-simple', 'en-simple'])
new_associations_df.to_csv("../data/stparkpap/pap-en(extracted_words).csv")